In [24]:
import numpy as np
import cv2
import os
import subprocess
import matplotlib.pyplot as plt
import BitVector
import math

labels = ["beach", "building", "car", "mountain", "tree"]

def getTrainingImages():
    Paths, Imgs = ([], [])
    for l in labels:
        imLoc = "Images/training/%s/" %l
        out, _ = subprocess.Popen("ls "+imLoc, shell=True,
                                      stdout=subprocess.PIPE,
                                      stderr=subprocess.PIPE).communicate()
        Paths = Paths + [ImgDir + x.decode("utf-8") for x in out.split()]
        GT = [0]*20 + [1]*20 + [2]*20 + [3]*20 + [4]*20
        
    for Path in Paths:
        # Read images
        Img = cv2.imread(Path, cv2.IMREAD_GRAYSCALE) # (H, W) uint8 np ndarray
        Imgs.append(Img)
    
    return Imgs, Paths, GT

def getTestingImages():
    Paths, Imgs = ([], [])
    for l in labels:  
        cmd = "ls imagesDatabaseHW7/testing/%s_*" %l
        out, _  = subprocess.Popen(cmd, shell=True,
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE).communicate()
        Paths = Paths + [x.decode("utf-8") for x in out.split()]
    GT = [0]*5 + [1]*5 + [2]*5 + [3]*5 + [4]*5
    for Path in Paths:
        # Read images
        Img = cv2.imread(Path, cv2.IMREAD_GRAYSCALE) # (H, W) uint8 np ndarray
        Imgs.append(Img)
    return Imgs, Paths, GT # list of (2D uint8, str, int)

def LBP(Im):
    for i in (im.shape[0]):
        for j in (im.shape[1]):
            for p in range(8):
                du = math.cos(0.25*math.pi*p)
                dv = math.sin(0.25*math.pi*p)
                x = i + du; y = j + dv
                
getTestingImages()


([],
 [],
 [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4])

In [ ]:
import numpy as np
import cv2, os, time
from scipy import spatial
#========================================================================================================
def create_hist(img):
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hist = [0]*10
    P = 8
    R = 1
    for i in range(1, img.shape[0]-1):
        for j in range(1, img.shape[1]-1): #iterating through each pixel
            intensities = []
            for p in range(P):
                del_u = R*np.cos(2*np.pi*p/P)
                del_v = R*np.sin(2*np.pi*p/P)

                intensities.append((1-del_u)*(1-del_v)*img[i-1,j+1] + (1-del_u)*del_v*img[i+1,j+1] \
                                   + del_u*(1-del_v)*img[i-1,j-1] + del_u*del_v*img[i+1,j-1])
            intensities = np.array(intensities)
            intensities = (intensities >= img[i][j]).astype(int)
            #finding minimum value pattern
            min_val = np.packbits(intensities)
            for k in range(len(intensities)):
                pattern = list(map(lambda x: (x+k)%8, [0,1,2,3,4,5,6,7]))
                cur_val = np.packbits(intensities[pattern])
                if cur_val < min_val:
                    min_val = cur_val
            code = np.binary_repr(min_val.item(), width = 8)
            #analyzing runs
            if code[0] == '1': #if all are 1
                hist[8] += 1
            else:
                k = 7
                while(code[k] == code[k-1] and k > 0):
                    k-=1
                if k == 0: #if all are 0
                    hist[0] += 1
                else:
                    break1 = k
                    k-=1
                    while(code[k] == code[k-1] and k > 0):
                        k-=1
                    if k == 0: #if exactly 2 runs
                        hist[break1] += 1
                    else: #if more than 2 runs
                        hist[9] += 1
    assert sum(hist) == (img.shape[0]-2) * (img.shape[1]-2)
    return hist
#========================================================================================================
def train(className):
    '''appends to hists.txt the histograms for a particular class' training images'''
    hists = []
    for file in os.listdir('Images/training/{}/'.format(className)):
        img = cv2.imread('Images/training/{}/'.format(className) + file)
        start = time.time()
        hists.append(create_hist(img))
        print('img {} of {} took {} seconds'.format(file, className, time.time() - start))
    with open(path + 'hists.txt', 'a') as fh:
        for hist in hists:
            to_write = ''
            for i in range(len(hist) - 1):
                to_write += str(hist[i]) + ','
            to_write += str(hist[-1])
            fh.write(to_write + '\n')
#========================================================================================================
def classify(img):
    hist = create_hist(img)
    with open(path + 'hists.txt', 'r') as fh:
        lines = fh.readlines()
        dists = []
        i = 0
        for line in lines:
            cur_hist = list(map(lambda x: int(x), line.split(',')))
            dists.append((spatial.distance.euclidean(cur_hist, hist), int(i/20)))
            i+=1
        dists.sort()
        count = [0]*5
        for item in dists[:5]:
            count[item[1]] += 1
        prediction = count.index(max(count))
        return prediction     

def LoadTestImg():
    imgs = []
    for i in range(5):
        im = cv2.imread("Images/testing/beach_%d.jpg" %(i+1))
        imgs.append(cv2.cvtColor(im,cv2.COLOR_BGR2GRAY))
    for i in range(5):
        im = cv2.imread("Images/testing/building_%d.jpg" %(i+1))
        imgs.append(cv2.cvtColor(im,cv2.COLOR_BGR2GRAY))
    for i in range(5):
        im = cv2.imread("Images/testing/car_%d.jpg" %(i+1))
        imgs.append(cv2.cvtColor(im,cv2.COLOR_BGR2GRAY))
    for i in range(5):
        im = cv2.imread("Images/testing/mountain_%d.jpg" %(i+1))
        imgs.append(cv2.cvtColor(im,cv2.COLOR_BGR2GRAY))
    for i in range(5):
        im = cv2.imread("Images/testing/tree_%d.jpg" %(i+1))
        imgs.append(cv2.cvtColor(im,cv2.COLOR_BGR2GRAY))
    return imgs
    
train('beach')
train('building')
train('car')
train('mountain')
train('tree')

labels = ['beach', 'building', 'car', 'mountain', 'tree']
confusion = np.zeros((5,5))
i = 0
for file in os.listdir('Images/testing/'):
    img = cv2.imread('Images/testing/{}'.format(file))
    ans = classify(img)
    if labels[ans] == file[:-6]:
        print('{} is correctly classified as {}'.format(file, labels[ans]))
    else:
        print('{} is incorrectly classified as {}'.format(file, labels[ans]))
    confusion[int(i/5), ans] += 1
    i += 1
print('Confusion matrix: ', confusion)


In [40]:
import numpy as np
import cv2
import os
import subprocess
import matplotlib.pyplot as plt
import BitVector
import math

labels = ["beach", "building", "car", "mountain", "tree"]

def getCirclePts():
    pts = []
    for p in range(8):
        pts.append([np.cos(0.25*np.pi*p), np.sin(0.25*np.pi*p)])
    return pts

def LoadTrainImg():
    imgs = [[],[],[],[],[]]
    for i in range(20):
        im = cv2.imread("Images/training/beach/%d.jpg" %(i+1))
        imgs[0].append(cv2.cvtColor(im,cv2.COLOR_BGR2GRAY))
    for i in range(20):
        im = cv2.imread("Images/training/building/%d.jpg" %(i+1))
        imgs[1].append(cv2.cvtColor(im,cv2.COLOR_BGR2GRAY))
    for i in range(20):
        im = cv2.imread("Images/training/car/%d.jpg" %(i+1))
        imgs[2].append(cv2.cvtColor(im,cv2.COLOR_BGR2GRAY))
    for i in range(20):
        im = cv2.imread("Images/training/mountain/%d.jpg" %(i+1))
        imgs[3].append(cv2.cvtColor(im,cv2.COLOR_BGR2GRAY))
    for i in range(20):
        im = cv2.imread("Images/training/tree/%d.jpg" %(i+1))
        imgs[4].append(cv2.cvtColor(im,cv2.COLOR_BGR2GRAY))
    return imgs

def LoadTestImg():
    imgs = []
    for i in range(5):
        im = cv2.imread("Images/testing/beach_%d.jpg" %(i+1))
        imgs.append(cv2.cvtColor(im,cv2.COLOR_BGR2GRAY))
    for i in range(5):
        im = cv2.imread("Images/testing/building_%d.jpg" %(i+1))
        imgs.append(cv2.cvtColor(im,cv2.COLOR_BGR2GRAY))
    for i in range(5):
        im = cv2.imread("Images/testing/car_%d.jpg" %(i+1))
        imgs.append(cv2.cvtColor(im,cv2.COLOR_BGR2GRAY))
    for i in range(5):
        im = cv2.imread("Images/testing/mountain_%d.jpg" %(i+1))
        imgs.append(cv2.cvtColor(im,cv2.COLOR_BGR2GRAY))
    for i in range(5):
        im = cv2.imread("Images/testing/tree_%d.jpg" %(i+1))
        imgs.append(cv2.cvtColor(im,cv2.COLOR_BGR2GRAY))
    return imgs

def euclid(test, train):
    tot = 0
    for i in range(test):
        tot += (test[i] - train[i])**2
    return math.sqrt(tot)

def LBP(im):
    circle = getCirclePts()
    LBP = []
    for i in range(im.shape[0]):
        for j in range(im.shape[1]):
            pts = []
            for p in circle:
                pt = [i+p[0], j+p[1]]
                pts.append(pt)
            for p in pts:
                A = [p[0] + 1]
                B = pts[0]
                C = pts[2]
                D = pts[1]
            
train = LoadTrainImg()
trainingLPBBeach = []
for im in train[0]:
    trainingLPBBeach.append(LBP(im))
trainingLPBBuilding = []
for im in train[0]:
    trainingLPBBuilding.append(LBP(im))
trainingLPBCar = []
for im in train[0]:
    trainingLPBCar.append(LBP(im))
trainingLPBMountain = []
for im in train[0]:
    trainingLPBMountain.append(LBP(im))
trainingLPBTree = []
for im in train[0]:
    trainingLPBTree.append(LBP(im))
    
test = LoadTestImg()
for im in test:
    testLBP = LBP(im)
    minDist = 99999
    bestLabel = ""
    for i in trainingLPBBeach:
        dist = euclid(testLBP, i)
        if (dist < minDist):
            minDist = dist
            bestLabel = "Beach"
    for i in trainingLPBBuilding:
        dist = euclid(testLBP, i)
        if (dist < minDist):
            minDist = dist
            bestLabel = "Building"
    for i in trainingLPBCar:
        dist = euclid(testLBP, i)
        if (dist < minDist):
            minDist = dist
            bestLabel = "Car"
    for i in trainingLPBMountain:
        dist = euclid(testLBP, i)
        if (dist < minDist):
            minDist = dist
            bestLabel = "Mountain"
    for i in trainingLPBBuilding:
        dist = euclid(testLBP, i)
        if (dist < minDist):
            minDist = dist
            bestLabel = "Tree"
    print(bestLabel)

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-wwma2wne\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
